In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
import os
import time
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from torchvision.models import resnet50
import torch.nn as nn
import torch.optim as optim



In [42]:

# 1. Data Loading
def load_data(data_dir, batch_size, num_workers):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resizing the image
        transforms.ToTensor(),  # Convert to tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization
    ])

    dataset = ImageFolder(data_dir, transform=transform)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    return train_loader, test_loader

In [43]:
# 2. Model Definition
def build_model(num_classes):
    model = resnet50(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model


In [44]:
# 3. Model Training
def train_model(model, train_loader, criterion, optimizer, device, epochs):
    model.train()
    model.to(device)

    for epoch in range(epochs):
        running_loss = 0.0
        start_time = time.time()  # Start timing
        step = 0  # Initialize step counter

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            step += 1  # Increment step counter

            # Print every 20 steps
            if step % 20 == 0:
                print(f'Epoch {epoch+1}, Step {step}, Intermediate Loss: {loss.item()}')

        end_time = time.time()  # End timing
        epoch_time = end_time - start_time  # Calculate epoch duration
        average_loss = running_loss / len(train_loader)

        print(f'Epoch {epoch+1}, Average Loss: {average_loss}, Epoch Time: {epoch_time:.2f} seconds')


In [45]:
# 4. Model Evaluation
def evaluate_model(model, test_loader, device):
    model.eval()
    model.to(device)
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy: {100 * correct / total}%')


In [46]:
# 5. Performance Optimization and Main Execution
def main():
    data_dir = '/content/drive/MyDrive/JPEGImages'
    batch_size = 32
    num_workers = 4
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    num_classes = 50
    epochs = 10

    train_loader, test_loader = load_data(data_dir, batch_size, num_workers)
    model = build_model(num_classes)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    train_model(model, train_loader, criterion, optimizer, device, epochs)
    evaluate_model(model, test_loader, device)

if __name__ == "__main__":
    main()



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Step 20, Intermediate Loss: 3.5155599117279053
Epoch 1, Step 40, Intermediate Loss: 3.0217132568359375
Epoch 1, Step 60, Intermediate Loss: 2.9732985496520996
Epoch 1, Step 80, Intermediate Loss: 2.755629539489746
Epoch 1, Step 100, Intermediate Loss: 2.6672255992889404
Epoch 1, Step 120, Intermediate Loss: 3.093343496322632
Epoch 1, Step 140, Intermediate Loss: 2.2943475246429443
Epoch 1, Step 160, Intermediate Loss: 2.1358838081359863
Epoch 1, Step 180, Intermediate Loss: 2.773421049118042
Epoch 1, Step 200, Intermediate Loss: 1.6024249792099
Epoch 1, Step 220, Intermediate Loss: 2.2351114749908447
Epoch 1, Step 240, Intermediate Loss: 2.0748350620269775
Epoch 1, Average Loss: 2.680093662683354, Epoch Time: 41.01 seconds
Epoch 2, Step 20, Intermediate Loss: 1.9613099098205566
Epoch 2, Step 40, Intermediate Loss: 1.7018483877182007
Epoch 2, Step 60, Intermediate Loss: 1.5175178050994873
Epoch 2, Step 80, Intermediate Loss: 1.5747443437576294
Epoch 2, Step 100, Intermediate Lo